# **Data pipeline**
<a target="_blank" href="https://colab.research.google.com/github/raphaelrubrice/scVAE_mva2025/blob/raph/data_pipeline.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## **Colab setup**

In [ ]:
!git clone https://github.com/raphaelrubrice/scVAE_mva2025.git
%cd scVAE_mva2025
!python -m pip install -e .

### Test Downloads

In [2]:
from data_pipeline.src.downloader import run_downloads

run_downloads()


Saving extracted PBMC datasets to: /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw

⇩ Downloading and extracting to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34


CD34: 100%|██████████| 38.0M/38.0M [00:19<00:00, 1.99MB/s]


✓ Extracted to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34
✓ Finished CD34
------------------------------------------------------------
⇩ Downloading and extracting to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B


CD19_B: 100%|██████████| 18.0M/18.0M [00:13<00:00, 1.35MB/s]


✓ Extracted to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B
✓ Finished CD19_B
------------------------------------------------------------
⇩ Downloading and extracting to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK


...

✓ Extracted to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD45RO
✓ Finished CD4_CD45RO
------------------------------------------------------------
⇩ Downloading and extracting to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8


CD8: 100%|██████████| 20.0M/20.0M [00:07<00:00, 2.57MB/s]


✓ Extracted to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8
✓ Finished CD8
------------------------------------------------------------
⇩ Downloading and extracting to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA


CD8_CD45RA: 100%|██████████| 20.9M/20.9M [00:14<00:00, 1.44MB/s]


✓ Extracted to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA
✓ Finished CD8_CD45RA
------------------------------------------------------------
All PBMC datasets ready in: /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw


In [4]:
from data_pipeline.src.config import DATASETS
from data_pipeline.src.load_anndata import load_anndata
from pathlib import Path

folder = Path("data_pipeline/data/pbmc_raw/CD4_CD45RO")
meta = DATASETS["CD4_CD45RO"]

adata = load_anndata(folder, meta)
adata

Loading 10x data from: data_pipeline/data/pbmc_raw/CD4_CD45RO/filtered_matrices_mex/hg19
✓ Loaded 10224 cells × 32738 genes from CD4_CD45RO


AnnData object with n_obs × n_vars = 10224 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'

In [5]:
from data_pipeline.src.combine import run_combine

combined, collection = run_combine(
    do_write_shards=True,
    write_combined=True,
    harmonize_var=False
)

Loading all PBMC AnnData objects...
📦 Loading dataset: CD34
Loading 10x data from: /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34/filtered_matrices_mex/hg19
✓ Loaded 9232 cells × 32738 genes from CD34
📦 Loading dataset: CD19_B
Loading 10x data from: /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B/filtered_matrices_mex/hg19
✓ Loaded 10085 cells × 32738 genes from CD19_B
📦 Loading dataset: CD56_NK
Loading 10x data from: /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK/filtered_matrices_mex/hg19
✓ Loaded 8385 cells × 32738 genes from CD56_NK
📦 Loading dataset: CD4_helper
Loading 10x data from: /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_helper/filtered_matrices_mex/hg19
✓ Loaded 11213 cells × 32738 genes from CD4_helper
📦 Loading dataset: CD4_CD25
Loading 10x data from: /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD25/filtered_matrices_mex/hg19
✓ Loaded 10263 cells × 32738 genes from 

/home/phalafail/MVA/scVAE_mva2025/.venv/lib/python3.12/site-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


✓ Combined AnnData written to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_processed/pbmc_combined.h5ad
✓ Label maps saved to /home/phalafail/MVA/scVAE_mva2025/data_pipeline/data/pbmc_processed/label_maps.json
Building in‑memory AnnCollection...


/home/phalafail/MVA/scVAE_mva2025/.venv/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ AnnCollection constructed from 9 shards.

Combine stage complete!



In [6]:
import scanpy as sc
adata_combined = sc.read_h5ad("data_pipeline/data/pbmc_processed/pbmc_combined.h5ad")
adata_combined

/home/phalafail/MVA/scVAE_mva2025/.venv/lib/python3.12/site-packages/anndata/_core/anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 92043 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'
    uns: 'label_maps'

In [ ]:
from data_pipeline.src.dataloader import build_cv_dataloaders

folds, test_loader = build_cv_dataloaders(
    shard_dir="data_pipeline/data/pbmc_processed/shards",
    label_maps_path="data_pipeline/data/pbmc_processed/label_maps.json",
    batch_size=256,
    one_hot=True,      
)

# **Cross-Validation of architectures**

In [ ]:
import torch
from mixture_vae.distributions import NormalDistribution, UniformDistribution, NegativeBinomial

## **Cross-Validated scVAE**

In [ ]:
from mixture_vae.mvae import MixtureVAE

In [ ]:
# Problem setup
input_dim = len(KEPT_GENES) # genes
hidden_dim = 256 # hidden neurons per layer
n_components = 9 # 9 clusters are assumed
latent_dim = 32 # dimension latent space
n_layers = 3 # number of encoding and decoding layers

# Prior on latent: Standard Gaussian in R2
mu = torch.zeros((1,latent_dim))
std = torch.ones((1,latent_dim))
prior_latent = NormalDistribution({"mu":mu,
                                    "std":std})

# Prior on input gene counts: NB for each gene 
p = 0.5 * torch.ones((1,input_dim)) # 50/50 chance of expression
r = torch.mean(X, dim=0).reshape(1,-1) # prior = average count in train data
prior_input = NegativeBinomial({"p":p,
                                "r":r})

# Prior on cluster repartitions (mixture): Assume balanced 
# cluster classes = Uniform on [0,1]
a = torch.zeros((1,n_components))
b = torch.ones((1,n_components))
prior_categorical = UniformDistribution({"a":a, 
                                          "b":b})

# Posterior on latent: Gaussian on R2 
# (here assumed posterior = assumed prior 
# but it could have been differnet)
mu = torch.zeros((1,latent_dim))
std = torch.ones((1,latent_dim))
posterior_latent = NormalDistribution({"mu":mu,
                                        "std":std})

In [ ]:
cv_scVAE = []
for fold in tqdm(list(range(len(folds)))):
    print(f"\nInstantiating scVAE..")
    scVAE = MixtureVAE(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            n_components=n_components,
            n_layers=n_layers,
            prior_latent=prior_latent,
            prior_input=prior_input,
            prior_categorical=prior_categorical,
            posterior_latent=posterior_latent
        )
    
    train_loader, val_loader = folds[fold]

    optimizer = torch.optim.Adam(scVAE.parameters(), lr=1e-3)

    EPOCHS = 50
    BETA_KL = 0.5
    WARMUP_BETA = int(0.2*EPOCHS)
    PATIENCE = 5
    TOL = 5e-3

    scVAE, losses, parts, clusters, all_betas = training_mvae(
        train_loader,
        val_loader,
        scVAE,
        optimizer,
        epochs=EPOCHS,
        beta_kl=BETA_KL,
        warmup=WARMUP_BETA,
        patience=PATIENCE,
        tol=TOL,
        show_loss_every=1,
        model_type=0,
        track_clusters=True,
    )
    
    cv_scVAE.append(scVAE)
    # plot training and validation losses
    plot_loss_components(parts["train"], 
                         parts["val"], 
                         all_betas, 
                         title="Loss Breakdown",
                         save_path=f"./scVAE_{fold}_losses.pdf")



## **Cross-Validated independent Mixture of Mixtures**

In [ ]:
from mixture_vae.mvae import ind_MoMVAE

In [ ]:
cv_IndMoM = []
for fold in tqdm(list(range(len(folds)))):
    print(f"\nInstantiating IndMoM..")
    IndMoM = ind_MoMVAE(
            PARAMS = [
            {"input_dim": input_dim,
            "hidden_dim": hidden_dim,
            "n_components": n_components,
            "n_layers": n_layers,
            "prior_latent": prior_latent,
            "prior_input": prior_input,
            "prior_categorical": prior_categorical,
            "posterior_latent": posterior_latent} 
            for n_components in [2, 3, 5, 9]]
        )
    
    train_loader, val_loader = folds[fold]

    optimizer = torch.optim.Adam(IndMoM.parameters(), lr=1e-3)

    EPOCHS = 50
    BETA_KL = 0.5
    WARMUP_BETA = int(0.2*EPOCHS)
    PATIENCE = 5
    TOL = 5e-3

    IndMoM, losses, parts, clusters, all_betas = training_mvae(
        train_loader,
        val_loader,
        IndMoM,
        optimizer,
        epochs=EPOCHS,
        beta_kl=BETA_KL,
        warmup=WARMUP_BETA,
        patience=PATIENCE,
        tol=TOL,
        show_loss_every=1,
        model_type=1,
        track_clusters=True,
    )
    cv_IndMoM.append(IndMoM)
    # plot training and validation losses
    plot_loss_components(parts["train"], 
                         parts["val"], 
                         all_betas, 
                         title="Loss Breakdown",
                         save_path=f"./IndMoM_{fold}_losses.pdf")



## **Cross-Validated Mixture of Mixtures**

In [ ]:
from mixture_vae.mvae import MoMixVAE
from mixture_vae.training import training_momixvae

In [ ]:
# Prior on cluster repartitions (mixture): Assume balanced 
# cluster classes = Uniform on [0,1]
# currently we assume the same for all levels
all_prior_categorical = []
for n_components in hierarchy_components:
    a = torch.zeros((1,n_components))
    b = torch.ones((1,n_components))
    prior_categorical = UniformDistribution({"a":a, 
                                            "b":b})
    all_prior_categorical.append(prior_categorical)

# Posterior on latent: Gaussian on R2 
# (here assumed posterior = assumed prior 
# but it could have been differnet)
# currently we assume the same for all levels
all_posterior_latent = []
for n_components in hierarchy_components:
    mu = torch.zeros((1,latent_dim))
    std = torch.ones((1,latent_dim))
    posterior_latent = NormalDistribution({"mu":mu,
                                        "std":std})
    all_posterior_latent.append(posterior_latent)

In [ ]:
cv_MoMix = []
for fold in tqdm(list(range(len(folds)))):
    print(f"\nInstantiating MoMix..")
    MoMix = MoMixVAE(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        hierarchy_components=hierarchy_components,
        n_layers=n_layers,
        prior_latent=prior_latent,
        prior_input=prior_input,
        all_prior_categorical=all_prior_categorical,
        all_posterior_latent=all_posterior_latent
    )
    
    train_loader, val_loader = folds[fold]

    optimizer = torch.optim.Adam(MoMix.parameters(), lr=1e-3)

    EPOCHS = 50
    BETA_KL = 0.5
    WARMUP_BETA = int(0.2*EPOCHS)
    PATIENCE = 5
    TOL = 5e-3

    MoMix, losses, parts, clusters, all_betas = training_momixvae(
        dataloader,
        val_dataloader,
        model,
        optimizer,
        epochs=EPOCHS,
        beta_kl=BETA_KL,
        warmup=WARMUP,
        patience=PATIENCE,
        tol=TOL,
        show_loss_every=1,
        track_clusters=True,
    )
    cv_MoMix.append(MoMix)
    # plot training and validation losses
    plot_loss_components(parts["train"], 
                         parts["val"], 
                         all_betas, 
                         title="Loss Breakdown",
                         save_path=f"./MoMix_{fold}_losses.pdf")



In [ ]:
from data_pipeline.src.dataloader import build_dataloaders

train_loader, val_loader, test_loader = build_dataloaders(
    shard_dir="data_pipeline/data/pbmc_processed/shards",
    label_maps_path="data_pipeline/data/pbmc_processed/label_maps.json",
    batch_size=512,
    one_hot=True,      
)

for i, batch in enumerate(train_loader):
        print("Example batch shapes:")
        print("X:", batch["X"].shape, "y1:", batch["y1"].shape, "y4:", batch["y4"].shape)
        break

/home/phalafail/MVA/scVAE_mva2025/.venv/lib/python3.12/site-packages/legacy_api_wrap/__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ Built AnnCollection with 92043 total cells.
Split: train=74554, val=8283, test=9206
✓ Train=74554, Val=8283, Test=9206
